In [28]:
import gurobipy as gp

In [29]:
import numpy as np
import numpy.random as npr
npr.seed = 1
N = 100 #25 50 75 100
v = np.zeros(N)
# v = npr.uniform(0.5, 1.5, N)
for i in range(N):
    utility = npr.randint(0, 2)*i*0.05 + npr.rand()*0.1
    v[i] = max(round(utility, 2), 0.01) 
print(v)

[0.09 0.06 0.11 0.23 0.25 0.33 0.35 0.44 0.08 0.47 0.02 0.06 0.02 0.72
 0.04 0.75 0.05 0.05 0.05 0.08 0.08 0.07 1.11 0.07 1.28 0.08 1.4  0.04
 1.49 1.45 0.03 1.62 1.67 0.03 0.08 1.85 1.82 0.09 1.98 1.99 0.07 0.07
 2.12 2.21 0.1  2.34 0.02 0.03 0.01 2.48 0.08 2.61 0.08 0.05 2.76 2.78
 2.8  2.92 2.93 3.03 0.01 0.01 3.16 0.04 0.07 3.33 0.07 3.37 3.43 3.47
 3.57 0.08 0.1  0.04 0.02 3.84 0.02 0.07 0.03 0.01 0.02 0.01 4.13 4.15
 4.26 0.07 0.08 0.03 4.43 4.5  0.01 4.57 4.61 4.72 4.71 0.01 0.04 4.86
 0.01 0.02]


In [30]:
def profit_mnl(p, S):
    global v
    pi = 0
    Vj_S = 0
    num = 0
    for i in S:
        num += p[i]*v[i]
        Vj_S += v[i]
    pi += num/(1+Vj_S)
    return pi

In [31]:
def nested_by_price_mnl(p):
    global N
    profits = [0 for i in range(N)]
    S = []
    for i in range(N):
        S.append(i)
        profits[i] = profit_mnl(p, S)
    best_S = [i for i in range(np.argmax(profits)+1)]
    return max(profits), best_S

In [32]:
def find_best_option_mnl(p, S):
    global v, N
    current_best = profit_mnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mnl(p, newS)
    return current_best, bestS, flag

def greedy_mnl(p):
    global v, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [33]:
import matplotlib.pyplot as plt
import math
import statistics as st
import time

In [34]:
n_simulations = 1
avg_price = 12

#compute prices
psimn = [0 for k in range(n_simulations)]
psimu = [0 for k in range(n_simulations)]
psime = [0 for k in range(n_simulations)]
for k in range(n_simulations):
    psimn[k] = abs(npr.normal(avg_price, avg_price, N))
    psimu[k] = npr.uniform(avg_price - math.sqrt(3*avg_price), avg_price + math.sqrt(3*avg_price), N)
    psime[k] = npr.exponential(avg_price, N)
    psimn[k] = np.round(psimn[k], 2)
    psimu[k] = np.round(psimu[k], 2)
    psime[k] = np.round(psime[k], 2)
    psimn[k] = sorted(psimn[k], reverse=True)
    psimu[k] = sorted(psimu[k], reverse=True)
    psime[k] = sorted(psime[k], reverse=True)

In [35]:
print(psimn)
print(psimu)
print(psime)

[[40.76, 39.35, 38.64, 35.36, 34.29, 31.07, 31.03, 30.73, 30.09, 29.72, 28.81, 28.8, 28.15, 27.33, 27.24, 26.93, 26.78, 26.36, 25.48, 24.84, 23.59, 22.02, 21.75, 21.74, 21.6, 21.58, 21.46, 20.81, 20.54, 20.21, 20.01, 19.96, 19.84, 19.31, 18.16, 18.01, 16.81, 16.6, 16.39, 16.01, 15.54, 15.36, 14.96, 14.64, 14.58, 14.55, 13.72, 13.51, 13.44, 13.17, 12.97, 12.89, 12.54, 12.45, 12.33, 12.05, 11.72, 11.31, 10.75, 9.75, 9.6, 9.4, 9.27, 8.1, 8.05, 7.98, 7.94, 7.65, 7.59, 7.57, 7.2, 7.0, 6.83, 6.07, 5.83, 5.83, 5.41, 5.2, 5.05, 4.24, 4.09, 4.0, 3.99, 3.9, 3.75, 3.46, 3.05, 2.85, 2.83, 2.81, 2.56, 1.58, 1.5, 1.41, 1.39, 1.17, 1.16, 0.69, 0.57, 0.42]]
[[17.89, 17.85, 17.83, 17.79, 17.78, 17.78, 17.72, 17.59, 17.49, 17.44, 17.44, 17.38, 17.27, 17.25, 17.25, 17.15, 16.93, 16.81, 16.81, 16.78, 16.73, 16.61, 16.6, 16.34, 16.29, 16.28, 16.16, 15.71, 15.53, 15.48, 15.43, 15.27, 15.02, 14.95, 14.94, 14.92, 14.73, 14.5, 14.47, 14.46, 14.23, 14.2, 14.1, 14.06, 13.89, 13.71, 13.62, 13.47, 13.29, 13.1, 13.

In [36]:
# problem solved using the nested formulation
profits_nested = []
times_nested = []
bestS_nested = []

for k in range(n_simulations):

    start_time = time.time()
    profit, bestS = nested_by_price_mnl(psim[k])
    end_time = time.time()

    profits_nested.append(profit)
    times_nested.append(end_time-start_time)
    bestS_nested.append(bestS)
    
avg_time_nested = st.mean(times_nested)

In [37]:
# problem solved using the greedy formulation
profits_greedy_n = []
times_greedy_n = []
bestS_greedy_n = []
profits_greedy_u = []
times_greedy_u = []
bestS_greedy_u = []
profits_greedy_e = []
times_greedy_e = []
bestS_greedy_e = []

for k in range(n_simulations):
    
    start_time = time.time()
    profitn, bestSn = greedy_mnl(psimn[k])
    profitu, bestSu = greedy_mnl(psimu[k])
    profite, bestSe = greedy_mnl(psime[k])
    end_time = time.time()
    
    profits_greedy_n.append(profitn)
    times_greedy_n.append(end_time-start_time)
    bestS_greedy_n.append(bestSn)
    profits_greedy_u.append(profitu)
    times_greedy_u.append(end_time-start_time)
    bestS_greedy_u.append(bestSu)
    profits_greedy_e.append(profite)
    times_greedy_e.append(end_time-start_time)
    bestS_greedy_e.append(bestSe)

# avg_time_greedy = st.mean(times_greedy)

In [40]:
# counting how many instances are different
count = 0
different_results_greedy = []

for k in range(n_simulations):
    if bestS_nested[k] != bestS_greedy_n[k]:
        count += 100/n_simulations
        different_results_greedy.append(100*profits_greedy_n[k]/profits_nested[k])
        print(bestS_nested[k], bestS_greedy_n[k], profits_nested[k], profits_greedy_n[k])
        print(bestS_nested[k], bestS_greedy_u[k], profits_nested[k], profits_greedy_u[k])
        print(bestS_nested[k], bestS_greedy_e[k], profits_nested[k], profits_greedy_e[k])
        print(len(bestS_nested[k]), len(bestS_greedy_n[k]))
        print(len(bestS_nested[k]), len(bestS_greedy_u[k]))
        print(len(bestS_nested[k]), len(bestS_greedy_e[k]))


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 26] 14.48769230769231 23.727759398496246
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 35] 14.48769230769231 15.172465233881159
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15] 14.48769230769231 28.28839357429718
31 21
31 33
31 15


In [39]:
if different_results_greedy != []:
    print('DR =  ' + str(count))
    print('Min = ' + str(min(different_results_greedy)))
    print('Mean = ' + str(st.mean(different_results_greedy)))
else:
    print('DR =  0')
    print('Min = NA')
    print('Mean = NA')
print('t = ' + str(100*avg_time_greedy/avg_time_nested))

DR =  100.0
Min = 163.778736423729
Mean = 163.778736423729
t = 514.904049370192
